# Sistema de classificação de sentimentos de críticas de produtos vendidos pela Amazon

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from urllib.request import urlopen
from sklearn.svm import SVC
from textblob import TextBlob
!pip install streamlit
import streamlit as st

In [4]:
# Baixe os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import json
import gzip
from urllib.request import urlopen

In [7]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [8]:
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  data = {}
  for d in parse(path):
    data[i] = d
    i += 1
  return pd.DataFrame.from_dict(data, orient='index')

# Carregue o conjunto de dados de críticas de filmes
data = getDF('/content/drive/MyDrive/Movies_and_TV_5.json.gz')

In [9]:
data.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"11 9, 2012",A2M1CU2IRZG0K9,0005089549,{'Format:': ' VHS Tape'},Terri,So sorry I didn't purchase this years ago when...,Amazing!,1352419200,NaN,NaN
1,5.0,True,"12 30, 2011",AFTUJYISOFHY6,0005089549,{'Format:': ' VHS Tape'},Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,1325203200,NaN,NaN
2,5.0,True,"04 21, 2005",A3JVF9Y53BEOGC,000503860X,{'Format:': ' DVD'},Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,1114041600,11,NaN
3,5.0,True,"04 6, 2005",A12VPEOEZS1KTC,000503860X,{'Format:': ' DVD'},JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,1112745600,5,NaN
4,5.0,True,"12 3, 2010",ATLZNVLYKP9AZ,000503860X,{'Format:': ' DVD'},T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,1291334400,5,NaN


In [10]:
data_review = data.head(10001)

In [11]:
data_review = data_review.drop(columns=['asin', 'style', 'unixReviewTime', 'vote', 'image', 'reviewTime'])

In [12]:
data_review.head()

,overall,verified,reviewerID,reviewerName,reviewText,summary
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band
3,5.0,True,A12VPEOEZS1KTC,JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!
4,5.0,True,ATLZNVLYKP9AZ,T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it


In [13]:
data_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10001 entries, 0 to 10000
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   overall       10001 non-null  float64
 1   verified      10001 non-null  bool   
 2   reviewerID    10001 non-null  object 
 3   reviewerName  10001 non-null  object 
 4   reviewText    9997 non-null   object 
 5   summary       10001 non-null  object 
dtypes: bool(1), float64(1), object(4)
memory usage: 478.6+ KB


In [14]:
data_review.reviewText = data_review.reviewText.astype(str)

In [15]:
data_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10001 entries, 0 to 10000
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   overall       10001 non-null  float64
 1   verified      10001 non-null  bool   
 2   reviewerID    10001 non-null  object 
 3   reviewerName  10001 non-null  object 
 4   reviewText    10001 non-null  object 
 5   summary       10001 non-null  object 
dtypes: bool(1), float64(1), object(4)
memory usage: 478.6+ KB


In [16]:
# Carregue o conjunto de dados de críticas de filmes

# data = pd.read_csv('movie_reviews.csv')

# Pré-processamento de texto
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

data_review['preprocessed_text'] = data_review['reviewText'].apply(preprocess_text)

In [17]:
data_review

,overall,verified,reviewerID,reviewerName,reviewText,summary,preprocessed_text
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!,sorry purchase years ago first came good enter...
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,believe tell receive blessing watching video c...
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,seen x live many times early days recent reuni...
3,5.0,True,A12VPEOEZS1KTC,JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,excited finally live concert video x seen twic...
4,5.0,True,ATLZNVLYKP9AZ,T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,x one best punk bands ever even like calling p...
...,...,...,...,...,...,...,...
9996,3.0,False,A1X054KUYG5V,One-Line Film Reviews,The Bottom Line:\n\nRomanced all out of propor...,2 stars out of 4,bottom line romanced proportion monty python h...
9997,3.0,True,A2V2UCJGF8C4ZT,Amazon Customer,"I was really looking forward to this movie, gi...",A disappointment,really looking forward movie given great thing...
9998,5.0,True,A1Z2HOY56LNIQA,Ronald Brezenski,"Monty Python, alone is more fun than whip crea...",Holy Grail of Fun & More,monty python alone fun whip cream oreo cookie ...
9999,5.0,True,A3OXHLG6DIBRW8,C. Hill,"Somewhat of a guilty pleasure, ""Monty Python a...","""It's Not the Real Grail?"" - Highly Recommended!",somewhat guilty pleasure monty python holy gra...


In [18]:
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00


In [19]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
classifier = pipeline('sentiment-analysis', model=model_name)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [20]:
name_lengths = data_review['reviewText'].str.len()

In [21]:
threshold_length = 512
filtered_data = data_review[name_lengths < threshold_length]

In [22]:
data_review['ResultadoHF'] = filtered_data.reviewText.apply(classifier)

In [23]:
data_review

,overall,verified,reviewerID,reviewerName,reviewText,summary,preprocessed_text,ResultadoHF
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!,sorry purchase years ago first came good enter...,"[{'label': 'POSITIVE', 'score': 0.999439895153..."
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,believe tell receive blessing watching video c...,"[{'label': 'POSITIVE', 'score': 0.999785006046..."
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,seen x live many times early days recent reuni...,"[{'label': 'POSITIVE', 'score': 0.999763309955..."
3,5.0,True,A12VPEOEZS1KTC,JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,excited finally live concert video x seen twic...,NaN
4,5.0,True,ATLZNVLYKP9AZ,T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,x one best punk bands ever even like calling p...,NaN
...,...,...,...,...,...,...,...,...
9996,3.0,False,A1X054KUYG5V,One-Line Film Reviews,The Bottom Line:\n\nRomanced all out of propor...,2 stars out of 4,bottom line romanced proportion monty python h...,"[{'label': 'NEGATIVE', 'score': 0.999030470848..."
9997,3.0,True,A2V2UCJGF8C4ZT,Amazon Customer,"I was really looking forward to this movie, gi...",A disappointment,really looking forward movie given great thing...,NaN
9998,5.0,True,A1Z2HOY56LNIQA,Ronald Brezenski,"Monty Python, alone is more fun than whip crea...",Holy Grail of Fun & More,monty python alone fun whip cream oreo cookie ...,"[{'label': 'POSITIVE', 'score': 0.999863505363..."
9999,5.0,True,A3OXHLG6DIBRW8,C. Hill,"Somewhat of a guilty pleasure, ""Monty Python a...","""It's Not the Real Grail?"" - Highly Recommended!",somewhat guilty pleasure monty python holy gra...,NaN


In [24]:
data_review_dropado = data_review.apply (pd.to_numeric, errors='coerce')
data_review_dropado = data_review.dropna()

In [25]:
data_review_dropado

,overall,verified,reviewerID,reviewerName,reviewText,summary,preprocessed_text,ResultadoHF
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!,sorry purchase years ago first came good enter...,"[{'label': 'POSITIVE', 'score': 0.999439895153..."
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,believe tell receive blessing watching video c...,"[{'label': 'POSITIVE', 'score': 0.999785006046..."
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,seen x live many times early days recent reuni...,"[{'label': 'POSITIVE', 'score': 0.999763309955..."
7,5.0,True,A2CFV9UPFTTM10,SuzieQ,The little ones love this,Love it,little ones love,"[{'label': 'POSITIVE', 'score': 0.999874949455..."
8,3.0,True,A3139J3877Y61F,SingingButterfly,Good songs. The DVD is a little boring.,Three Stars,good songs dvd little boring,"[{'label': 'NEGATIVE', 'score': 0.997869253158..."
...,...,...,...,...,...,...,...,...
9992,5.0,True,A2RRMTLWP3N6TB,W. Tappan Lum,Monty Python and the Holy Grail was the first ...,The result of a great comedic team,monty python holy grail first feature length w...,"[{'label': 'POSITIVE', 'score': 0.999884486198..."
9993,5.0,True,A18AWO90L4TXBP,Steve B.,There's a reason that this movie is legend: it...,Staggeringly creative!,reason movie legend outstandingly creative end...,"[{'label': 'POSITIVE', 'score': 0.999873995780..."
9995,5.0,True,A1E5E5QS5ZFYEU,Mr. Brian J. Devlin,I haven't seen this movie in quite some time a...,The Hunt For The Grail,seen movie quite time lost thing halarious,"[{'label': 'POSITIVE', 'score': 0.993808805942..."
9996,3.0,False,A1X054KUYG5V,One-Line Film Reviews,The Bottom Line:\n\nRomanced all out of propor...,2 stars out of 4,bottom line romanced proportion monty python h...,"[{'label': 'NEGATIVE', 'score': 0.999030470848..."


In [26]:
data_class = data_review_dropado

In [27]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].astype(str)

<ipython-input-27-2832ad3d87ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].astype(str)


In [28]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace(']', '')

<ipython-input-28-87069907d1ab>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace(']', '')
<ipython-input-28-87069907d1ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace(']', '')


In [29]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('[', '')

<ipython-input-29-47e6e66e2607>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('[', '')
<ipython-input-29-47e6e66e2607>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('[', '')


In [30]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('{', '')

<ipython-input-30-d276dea674ec>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('{', '')
<ipython-input-30-d276dea674ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('{', '')


In [31]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('}', '')

<ipython-input-31-820d4a1edcce>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('}', '')
<ipython-input-31-820d4a1edcce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace('}', '')


In [32]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace("'", '')

<ipython-input-32-4333d683aba3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace("'", '')


In [33]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace("label:", '')

<ipython-input-33-79fda9673de0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace("label:", '')


In [34]:
data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace("score:", '')

<ipython-input-34-db37a8208132>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['ResultadoHF'] = data_class['ResultadoHF'].str.replace("score:", '')


In [35]:
data_class.head()

,overall,verified,reviewerID,reviewerName,reviewText,summary,preprocessed_text,ResultadoHF
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!,sorry purchase years ago first came good enter...,"POSITIVE, 0.9994398951530457"
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,believe tell receive blessing watching video c...,"POSITIVE, 0.9997850060462952"
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,seen x live many times early days recent reuni...,"POSITIVE, 0.9997633099555969"
7,5.0,True,A2CFV9UPFTTM10,SuzieQ,The little ones love this,Love it,little ones love,"POSITIVE, 0.9998749494552612"
8,3.0,True,A3139J3877Y61F,SingingButterfly,Good songs. The DVD is a little boring.,Three Stars,good songs dvd little boring,"NEGATIVE, 0.9978692531585693"
...,...,...,...,...,...,...,...,...
9992,5.0,True,A2RRMTLWP3N6TB,W. Tappan Lum,Monty Python and the Holy Grail was the first ...,The result of a great comedic team,monty python holy grail first feature length w...,"POSITIVE, 0.9998844861984253"
9993,5.0,True,A18AWO90L4TXBP,Steve B.,There's a reason that this movie is legend: it...,Staggeringly creative!,reason movie legend outstandingly creative end...,"POSITIVE, 0.9998739957809448"
9995,5.0,True,A1E5E5QS5ZFYEU,Mr. Brian J. Devlin,I haven't seen this movie in quite some time a...,The Hunt For The Grail,seen movie quite time lost thing halarious,"POSITIVE, 0.9938088059425354"
9996,3.0,False,A1X054KUYG5V,One-Line Film Reviews,The Bottom Line:\n\nRomanced all out of propor...,2 stars out of 4,bottom line romanced proportion monty python h...,"NEGATIVE, 0.9990304708480835"


In [36]:
data_class[['Sentiment', 'Score']] = data_class.ResultadoHF.str.split(",", expand = True)

<ipython-input-36-4626f453f888>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class[['Sentiment', 'Score']] = data_class.ResultadoHF.str.split(",", expand = True)
<ipython-input-36-4626f453f888>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class[['Sentiment', 'Score']] = data_class.ResultadoHF.str.split(",", expand = True)


In [37]:
data_class.head()

,overall,verified,reviewerID,reviewerName,reviewText,summary,preprocessed_text,ResultadoHF,Sentiment,Score
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!,sorry purchase years ago first came good enter...,"POSITIVE, 0.9994398951530457",POSITIVE,0.9994398951530457
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,believe tell receive blessing watching video c...,"POSITIVE, 0.9997850060462952",POSITIVE,0.9997850060462952
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,seen x live many times early days recent reuni...,"POSITIVE, 0.9997633099555969",POSITIVE,0.9997633099555969
7,5.0,True,A2CFV9UPFTTM10,SuzieQ,The little ones love this,Love it,little ones love,"POSITIVE, 0.9998749494552612",POSITIVE,0.9998749494552612
8,3.0,True,A3139J3877Y61F,SingingButterfly,Good songs. The DVD is a little boring.,Three Stars,good songs dvd little boring,"NEGATIVE, 0.9978692531585693",NEGATIVE,0.9978692531585693


In [38]:
data_class['Sentiment'] = data_class['Sentiment'].str.replace('POSITIVE', 'Positivo').str.title()

<ipython-input-38-d1fe715b9dfd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['Sentiment'] = data_class['Sentiment'].str.replace('POSITIVE', 'Positivo').str.title()


In [39]:
data_class['Sentiment'] = data_class['Sentiment'].str.replace('NEGATIVE', 'Negativo').str.title()

<ipython-input-39-ac436cb4a492>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['Sentiment'] = data_class['Sentiment'].str.replace('NEGATIVE', 'Negativo').str.title()


In [40]:
data_class['Sentiment'] = data_class['Sentiment'].str.replace('Negative', 'Crítica Negativa').str.title()

<ipython-input-40-ef62babcd415>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['Sentiment'] = data_class['Sentiment'].str.replace('Negative', 'Crítica Positiva').str.title()


In [43]:
data_class['Sentiment'] = data_class['Sentiment'].str.replace('Positivo', 'Crítica Positiva').str.title()

<ipython-input-43-475f192e30c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_class['Sentiment'] = data_class['Sentiment'].str.replace('Positivo', 'Crítica Positiva').str.title()


In [44]:
data_class

,overall,verified,reviewerID,reviewerName,reviewText,summary,preprocessed_text,ResultadoHF,Sentiment,Score
0,5.0,True,A2M1CU2IRZG0K9,Terri,So sorry I didn't purchase this years ago when...,Amazing!,sorry purchase years ago first came good enter...,"POSITIVE, 0.9994398951530457",Crítica Positiva,0.9994398951530457
1,5.0,True,AFTUJYISOFHY6,Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,believe tell receive blessing watching video c...,"POSITIVE, 0.9997850060462952",Crítica Positiva,0.9997850060462952
2,5.0,True,A3JVF9Y53BEOGC,Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,seen x live many times early days recent reuni...,"POSITIVE, 0.9997633099555969",Crítica Positiva,0.9997633099555969
7,5.0,True,A2CFV9UPFTTM10,SuzieQ,The little ones love this,Love it,little ones love,"POSITIVE, 0.9998749494552612",Crítica Positiva,0.9998749494552612
8,3.0,True,A3139J3877Y61F,SingingButterfly,Good songs. The DVD is a little boring.,Three Stars,good songs dvd little boring,"NEGATIVE, 0.9978692531585693",Crítica Negativa,0.9978692531585693
...,...,...,...,...,...,...,...,...,...,...
9992,5.0,True,A2RRMTLWP3N6TB,W. Tappan Lum,Monty Python and the Holy Grail was the first ...,The result of a great comedic team,monty python holy grail first feature length w...,"POSITIVE, 0.9998844861984253",Crítica Positiva,0.9998844861984253
9993,5.0,True,A18AWO90L4TXBP,Steve B.,There's a reason that this movie is legend: it...,Staggeringly creative!,reason movie legend outstandingly creative end...,"POSITIVE, 0.9998739957809448",Crítica Positiva,0.9998739957809448
9995,5.0,True,A1E5E5QS5ZFYEU,Mr. Brian J. Devlin,I haven't seen this movie in quite some time a...,The Hunt For The Grail,seen movie quite time lost thing halarious,"POSITIVE, 0.9938088059425354",Crítica Positiva,0.9938088059425354
9996,3.0,False,A1X054KUYG5V,One-Line Film Reviews,The Bottom Line:\n\nRomanced all out of propor...,2 stars out of 4,bottom line romanced proportion monty python h...,"NEGATIVE, 0.9990304708480835",Crítica Negativa,0.9990304708480835


In [45]:
data_class.to_json('/content/sample_data/data_class.json')

In [48]:
# Vetorização de texto usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_class['preprocessed_text'])
y = data_class['Sentiment']

# Treinamento do modelo SVM
classifier = SVC(kernel='linear')
classifier.fit(X, y)

# Função para realizar a classificação de sentimentos
def analisar_sentimento(texto):
    preprocessed_text = preprocess_text(texto)
    text_vector = vectorizer.transform([preprocessed_text])
    sentiment = classifier.predict(text_vector)[0]
    return sentiment

In [49]:
# Exemplo de uso
texto = input("Digite sua crítica: ")
sentimento = analisar_sentimento(texto)
print(f"Texto: {texto}")
print(f"Sentimento: {sentimento}")

Digite sua crítica: Cool
Texto: Cool
Sentimento:  Positivo
